In [1]:
import pandas as pd
import sqlite3

# Load Step 2 outputs
fact_orders = pd.read_csv("data/cleaned/fact_orders.csv", parse_dates=["Order_Date","Month_End"])
dim_products = pd.read_csv("data/cleaned/dim_products.csv")
dim_customers = pd.read_csv("data/cleaned/dim_customers.csv", parse_dates=["First_Purchase","Last_Purchase"])
dim_date = pd.read_csv("data/cleaned/dim_date.csv", parse_dates=["Date","Month_End"])

# Create in-memory SQLite and register tables
conn = sqlite3.connect(":memory:")
fact_orders.to_sql("fact_orders", conn, index=False, if_exists="replace")
dim_products.to_sql("dim_products", conn, index=False, if_exists="replace")
dim_customers.to_sql("dim_customers", conn, index=False, if_exists="replace")
dim_date.to_sql("dim_date", conn, index=False, if_exists="replace")

# Helpful: create simple indexes
with conn:
    conn.execute("CREATE INDEX IF NOT EXISTS ix_fact_orders_date ON fact_orders(Order_Date);")
    conn.execute("CREATE INDEX IF NOT EXISTS ix_fact_orders_month ON fact_orders(Month_End);")
    conn.execute("CREATE INDEX IF NOT EXISTS ix_fact_orders_customer ON fact_orders(Customer_Name);")


In [2]:
# 1) Daily revenue trend (can be later aggregated by month in BI)
q_revenue_daily = """
SELECT 
  DATE(Order_Date) AS order_date,
  SUM(Revenue) AS total_revenue,
  SUM(Profit) AS total_profit,
  COUNT(DISTINCT Order_ID) AS orders
FROM fact_orders
GROUP BY DATE(Order_Date)
ORDER BY DATE(Order_Date);
"""

rev_daily = pd.read_sql(q_revenue_daily, conn)
rev_daily.head()


,order_date,total_revenue,total_profit,orders
0,2018-04-01,5318.0,-243.0,2
1,2018-04-03,2247.0,-158.0,2
2,2018-04-05,75.0,0.0,1
3,2018-04-06,137.0,19.0,2
4,2018-04-08,2953.0,-1456.0,1


In [3]:
# 2) Revenue by month and category
# Use details-level category via bridging from Step 2 if needed; here we only have order-level.
# For simplicity, aggregate order revenue by month; category split will come from details if modeled later.
q_revenue_by_month = """
SELECT 
  DATE(Month_End) AS month_end,
  SUM(Revenue) AS revenue,
  SUM(Profit) AS profit,
  COUNT(DISTINCT Order_ID) AS orders
FROM fact_orders
GROUP BY DATE(Month_End)
ORDER BY DATE(Month_End);
"""

rev_by_month = pd.read_sql(q_revenue_by_month, conn)
rev_by_month.head()


,month_end,revenue,profit,orders
0,2018-04-30,32726.0,-3960.0,44
1,2018-05-31,28545.0,-3584.0,31
2,2018-06-30,23658.0,-4970.0,30
3,2018-07-31,12966.0,-2138.0,31
4,2018-08-31,30899.0,-2180.0,31


In [4]:
# 3) Top customers by revenue
q_top_customers = """
SELECT 
  Customer_Name,
  SUM(Net_Revenue) AS revenue,
  SUM(Profit) AS profit,
  COUNT(DISTINCT Order_ID) AS orders
FROM fact_orders
GROUP BY Customer_Name
ORDER BY revenue DESC
LIMIT 20;
"""

top_customers = pd.read_sql(q_top_customers, conn)
top_customers.head()


,Customer_Name,revenue,profit,orders
0,Yaanvi,9177.0,488.0,2
1,Pooja,9030.0,-269.0,5
2,Abhishek,8135.0,1314.0,5
3,Surabhi,6889.0,412.0,4
4,Soumya,6869.0,894.0,3


In [5]:
# 4) Region performance: by State and City
q_region_perf = """
SELECT 
  State,
  City,
  SUM(Net_Revenue) AS revenue,
  SUM(Profit) AS profit,
  COUNT(DISTINCT Order_ID) AS orders,
  AVG(CASE WHEN Items > 0 THEN Revenue*1.0/Items END) AS avg_item_price
FROM fact_orders
GROUP BY State, City
ORDER BY revenue DESC;
"""

region_perf = pd.read_sql(q_region_perf, conn)
region_perf.head()


,State,City,revenue,profit,orders,avg_item_price
0,Madhya Pradesh,Indore,79069.0,4159.0,76,74.240947
1,Maharashtra,Mumbai,61867.0,1637.0,68,87.513071
2,Maharashtra,Pune,33481.0,4539.0,22,92.978443
3,Madhya Pradesh,Bhopal,23583.0,871.0,22,115.898108
4,Delhi,Delhi,22531.0,2987.0,22,83.474416


In [6]:
# 5) Repeat vs new customers by month
q_repeat_new = """
WITH cust_first AS (
  SELECT 
    Customer_Name,
    MIN(DATE(Order_Date)) AS first_date
  FROM fact_orders
  GROUP BY Customer_Name
),
orders_labeled AS (
  SELECT 
    f.Order_ID,
    DATE(f.Order_Date) AS order_date,
    DATE(f.Month_End) AS month_end,
    f.Customer_Name,
    f.Net_Revenue,
    CASE 
      WHEN DATE(f.Order_Date) = (SELECT first_date FROM cust_first c WHERE c.Customer_Name = f.Customer_Name)
      THEN 'New'
      ELSE 'Repeat'
    END AS cust_type
  FROM fact_orders f
)
SELECT 
  month_end,
  cust_type,
  SUM(Net_Revenue) AS revenue,
  COUNT(DISTINCT Order_ID) AS orders
FROM orders_labeled
GROUP BY month_end, cust_type
ORDER BY month_end, cust_type;
"""

repeat_new = pd.read_sql(q_repeat_new, conn)
repeat_new.head()


,month_end,cust_type,revenue,orders
0,2018-04-30,New,32726.0,44
1,2018-05-31,New,28545.0,31
2,2018-06-30,New,19375.0,27
3,2018-06-30,Repeat,4283.0,3
4,2018-07-31,New,11938.0,28


In [7]:
# 6) Simple customer RFM summary (uses dim_customers from Step 2)
q_customer_rfm = """
SELECT 
  Customer_Name,
  Recency_Days,
  Orders AS frequency,
  Monetary AS monetary,
  RFM_Segment
FROM dim_customers
ORDER BY monetary DESC
LIMIT 20;
"""

customer_rfm = pd.read_sql(q_customer_rfm, conn)
customer_rfm.head()


,Customer_Name,Recency_Days,frequency,monetary,RFM_Segment
0,Yaanvi,21,2,9177.0,R4F4M4
1,Pooja,35,5,9030.0,R4F4M4
2,Abhishek,114,5,8135.0,R3F4M4
3,Surabhi,201,4,6889.0,R1F4M4
4,Soumya,69,3,6869.0,R3F4M4


In [8]:
rev_daily.to_csv("data/cleaned/v_revenue_daily.csv", index=False)
rev_by_month.to_csv("data/cleaned/v_revenue_by_month.csv", index=False)
top_customers.to_csv("data/cleaned/v_top_customers.csv", index=False)
region_perf.to_csv("data/cleaned/v_region_performance.csv", index=False)
repeat_new.to_csv("data/cleaned/v_repeat_vs_new.csv", index=False)
customer_rfm.to_csv("data/cleaned/v_customer_rfm_top20.csv", index=False)

print("Saved SQL analytics outputs to data/cleaned/")


Saved SQL analytics outputs to data/cleaned/
